In [1]:
# ========================================
# Cell 1: 라이브러리 import 및 설정
# ========================================

import os
import sys
import glob
sys.path.append(os.path.abspath('../src'))

from config import (
    AEM_PAGES_MASTER_TXT, INPUT_DIR,
    AEM_BATCHES_TODO_DIR, PDF_BATCHES_TODO_DIR
)
from utils.batch_utils import create_text_file_batches, create_csv_file_batches

# 필요한 디렉토리 생성
os.makedirs(AEM_BATCHES_TODO_DIR, exist_ok=True)
os.makedirs(PDF_BATCHES_TODO_DIR, exist_ok=True)

print(f"📁 배치 출력 디렉토리:")
print(f"   - AEM: {AEM_BATCHES_TODO_DIR}")
print(f"   - PDF: {PDF_BATCHES_TODO_DIR}")

📁 배치 출력 디렉토리:
   - AEM: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/1_input/aem_batches_todo
   - PDF: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/1_input/pdf_batches_todo


In [ ]:
# ========================================
# Cell 2: AEM 페이지 목록 배치 생성
# ========================================

print("--- 1. AEM 페이지 목록 배치 생성 시작 ---")

# AEM 마스터 파일 존재 확인
if not os.path.exists(AEM_PAGES_MASTER_TXT):
    print(f"❌ AEM 페이지 마스터 파일을 찾을 수 없습니다: {AEM_PAGES_MASTER_TXT}")
else:
    create_text_file_batches(
        input_file=AEM_PAGES_MASTER_TXT,
        output_dir=AEM_BATCHES_TODO_DIR,
        batch_size=500
    )

print("-" * 50)

In [ ]:
# ========================================
# Cell 3: 다국어 PDF 목록 배치 생성
# ========================================

print("\n--- 2. 다국어 PDF 목록 배치 생성 시작 ---")

# pdf_list_master_*.csv 패턴의 모든 파일 찾기
pdf_master_files = glob.glob(os.path.join(INPUT_DIR, "pdf_list_master_*.csv"))

if not pdf_master_files:
    print("❌ pdf_list_master_*.csv 파일을 찾을 수 없습니다.")
    print(f"   확인 경로: {INPUT_DIR}")
else:
    print(f"🔍 발견된 PDF 매스터 파일들:")
    for file_path in pdf_master_files:
        print(f"   - {os.path.basename(file_path)}")
    
    print(f"\n📋 총 {len(pdf_master_files)}개 파일을 배치로 분할합니다...")
    
    for file_path in pdf_master_files:
        print(f"\n처리 중: {os.path.basename(file_path)}")
        create_csv_file_batches(
            input_file=file_path,
            output_dir=PDF_BATCHES_TODO_DIR,
            batch_size=500
        )

print("-" * 50)


In [ ]:
# ========================================
# Cell 4: 생성된 배치 파일 확인
# ========================================

print("\n--- 3. 생성된 배치 파일 확인 ---")

# AEM 배치 파일 확인
aem_batches = glob.glob(os.path.join(AEM_BATCHES_TODO_DIR, "aem_batch_*.list"))
print(f"📄 AEM 배치 파일: {len(aem_batches)}개")

# PDF 배치 파일 확인 (언어별)
pdf_batches = glob.glob(os.path.join(PDF_BATCHES_TODO_DIR, "pdf_batch_*.csv"))
print(f"📄 PDF 배치 파일: {len(pdf_batches)}개")

if pdf_batches:
    print("\n🌐 언어별 PDF 배치 파일:")
    language_counts = {}
    
    for batch_file in pdf_batches:
        filename = os.path.basename(batch_file)
        # pdf_batch_en_ko_001.csv에서 en_ko 추출
        parts = filename.split('_')
        if len(parts) >= 4:
            lang_pair = f"{parts[2]}_{parts[3]}"
            if lang_pair not in language_counts:
                language_counts[lang_pair] = 0
            language_counts[lang_pair] += 1
    
    for lang_pair, count in language_counts.items():
        source_lang, target_lang = lang_pair.split('_')
        print(f"   - {source_lang.upper()}-{target_lang.upper()}: {count}개 배치")

print(f"\n✅ 배치 생성 완료! 이제 데이터 수집을 시작할 수 있습니다.")